对于BSM微分方程:
$$
\frac{\partial f}{\partial t} + bS\frac{\partial f}{\partial S} + \frac{1}{2}\sigma^2S^2\frac{\partial^2 f}{\partial S^2} = rf \tag{5.1}
$$

香草期权可以获得其解析解，但是大部分期权很难通过解微分方程得到答案，因此，蒙特卡洛成为了价格模拟的一种方法，而除此之外，我们也可以使用有限差分方法来近似解。

有限差分法定价即通过有限差分的方法，解期权价值相关的偏微分方程，得到期权价值的数值解。这与微积分的切割思想相通，对于希腊字母而言：

$delta = \frac{\partial f}{\partial S} = \frac{f_{S+\Delta S}-f_{S-\Delta S}}{2\Delta S}$

$gamma = \frac{\partial^2 f}{\partial S^2} = \frac{f_{S+\Delta S}-2f_{S}+f_{S-\Delta S}}{\Delta S^2}$

$theta = \frac{\partial f}{\partial t} = \frac{f_{t+\Delta t}-f_{t-\Delta t}}{2\Delta t}$

上式解为有限差分法的体现。


构建t-S网格，i表示第i个时间点，j表示第j个价格点,即$S(i,j)=j\Delta S,t(i,j)=i\Delta t$，在空间维度上，即对S的差分基本采用中间差分法,而时间维度上，采取后向差分:

$$delta = \frac{f_{i,j+1}-f_{i,j-1}}{2\Delta S}$$

$$gamma = \frac{f_{i,j+1}-2f_{i,j}+f_{i,j-1}}{\Delta S^2}$$

$$
theta = \frac{f_{i,j}-f_{i-1,j}}{\Delta t}
$$

带入(5.1)，得:
$$
\frac{f_{i,j}-f_{i-1,j}}{\Delta t} + bj\Delta S\frac{f_{i,j+1}-f_{i,j-1}}{2\Delta S} + \frac{1}{2}\sigma^2j^2\Delta S^2\frac{f_{i,j+1}-2f_{i,j}+f_{i,j-1}}{\Delta S^2} = rf_{i-1,j}\\
\tag{5.2}
$$

整理得:
$$
f_{i-1,j} = a_jf_{i,j-1}+b_jf_{i,j}+c_jf_{i,j+1}\\
其中:\\
a_j = \frac{-0.5bj\Delta t + 0.5\sigma^2j^2\Delta t}{1+r\Delta t}\\
b_j = \frac{1-\sigma^2j^2\Delta t}{1+r\Delta t}\\
c_j = \frac{0.5bj\Delta t + 0.5\sigma^2j^2\Delta t}{1+r\Delta t} \tag{5.3}
$$


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
import pandas as pd
import scipy.stats as stats
import warnings
warnings.filterwarnings("ignore")
np.set_printoptions(suppress=True)
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #正常显示负号
@jit
def explicit_FD(CP,S,K,T,sigma,r,b,M,N):
    """
    显式有限差分法，是否收敛取决于dt，ds，相对来说不太稳定
               f[i+1,j+1]
    f[i,j] ⬅  f[i,j+1]
               f[i-1,j+1]

    Parameters
    ----------
    CP : "C","P"表示看涨或看跌
    S : 标的价格
    K : 行权价
    sigma:波动率
    r:无风险利率
    b:持有成本，当b = r 时，为标准的无股利模型，b=0时，为black76，b为r-q时，为支付股利模型，b为r-rf时为外汇期权.持有成本
    M : 初始给定的价格节点数量
    N: 初始给定的时间节点数量
    Returns
    期权价值
    """

    ds = S/M   # 确定价格步长，分子用S的意义在于可以让S必定落在网格点上，后续不需要使用插值法
    M = int(K/ds) *4  #确定覆盖的价格范围，这里设置为4倍的行权价，也可根据需要设置为其他，这里根据价格范围重新计算价格点位数量M
    S_idx= int(S / ds)   # S所在的index，用于方便确定初始S对应的期权价值
    dt = T/N  # 时间步长
    df = 1 / (1 + r*dt) #折现因子
    print(f"生产的网格：价格分为M = {M}个点位，时间分为N = {N}个点位")
    V_grid = np.zeros((M+1,N+1)) # 预先生成包括0在内的期权价值矩阵

    S_array = np.linspace(0, M*ds,M+1)  #价格序列
    T_array = np.linspace(0, N*dt,N+1)  #时间序列
    T2M_array = T_array[-1] -T_array    #生成到期时间的数组，方便后面计算边界条件

    if CP == "C":
        V_grid[:,N] = np.maximum(S_array - K,0)  #确定终值条件，到期时期权价值很好计算
        V_grid[M] = np.exp(-r*T2M_array) * (S_array[-1] * np.exp(b*T2M_array) - K)   # 上边界价格够高，期权表现像远期，这里是远期定价，而不是简单得S-X
    else:
        V_grid[:,N] = np.maximum(K - S_array ,0)  #确定终值条件，到期时期权价值很好计算
        V_grid[0] = np.exp(-r*T2M_array) * K


    for j in range(N-1,-1,-1): #时间倒推循环
        for i in range(1,M):  # 价格循环
            # 确定系数
            aj = 0.5 * (sigma**2 * i**2 - b*i) * dt
            bj = 1 - sigma**2 * i**2 * dt
            cj = 0.5 * (sigma**2 * i**2 + b*i) * dt
            # 显示也可以用矩阵来写，但是直接循环也比较直观
            V_grid[i,j] = df * (aj * V_grid[i-1,j+1] + bj * V_grid[i,j+1] + cj * V_grid[i+1,j+1])
            # 美式期权提前行权判断，欧式期权可以直接删除下面的代码
            if CP == "C":
                V_grid[i,j] = max(S_array[i] - K, V_grid[i,j])  #美式期权提前行权判断
            else:
                V_grid[i,j] = max(K - S_array[i], V_grid[i,j])
    return V_grid[S_idx,0] # 返回初0时点的初始价格的价值

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
explicit_FD(CP = "P",S = 36,K = 40,T = 0.5,sigma = 0.4,r = 0.06,b =0.06,M = 125,N=50000)

生产的网格：价格分为M = 552个点位，时间分为N = 50000个点位


5.9667421297107754